# Checkpoint Deep Dive

A checkpoint in DAG is to cache the execution state of a certain step. The purpose of a checkpoint may be one or some of the following:

* break the lineage
* avoid duplicated execution of the previous steps
* quickly recover the execution state and resume

## Spark Persist vs Checkpoint

In Spark, there are [persist](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence) and [checkpoint](https://jaceklaskowski.gitbooks.io/mastering-spark-sql/content/spark-sql-checkpointing.html) (different from [streaming checkpoint](https://spark.apache.org/docs/latest/streaming-programming-guide.html#checkpointing)) methods for rdd. 

Spark persist is to cache data into memory (or executor local disk), it does not break the lineage in Spark execution). It can't break the lineage because the assumption is that the storage either executor memory or local disk is not permanent. Some executor can die during execution, to retrieve that portion of data, the driver has to use the lineage to recompute and recache. This is a great and robust idea, and in most cases it works very well. However, when the lineage contains hundreds or thousands of steps, we may see stack overflow problems.

Spark checkpoint is to store data into file system, and it breaks the lineage. This is also a fine idea, because the data will not lose even the entire execution breaks. The drawback is speed and disk space usage. Practically, this is more useful for large data and expensive compute cases where the overhead of saving intermediate data into filesystem is ignorable compared to the cost of the compute itself. Another scenario is that the execution environment is not reliable, executors can die any time and the compute is expensive, in this case, breaking the lineage may significantly improve the performance and reliability.

It's worth to mention that Spark persist is lazy, only the first usage of the rdd will trigger the persist to happen. Spark checkpoint can be lazy or eager, eager means the caching operation happens immediately when requested. As of pyspark<=3.0.1, the checkpoint seems to be lazy by default, and it forces recompute of the previous steps unless you persist and checkpoint (see [this issue](https://issues.apache.org/jira/browse/SPARK-8582)). So the implementation itself if problematic.

Last but not the least, Spark persist's cache is for current execution only while Spark checkpoint's cache can be permanent (in reliable mode), so it can be retrived cross execution. This is similar to the caching mechanism of some workflow management solution such as Airflow. However practically, without a clear definition of determinism of the steps, retrieving checkpoints cross execution may cause more problem than it can solve.

## Generalizing The Concept

It is fair to say that Spark persist and checkpoint solves some common problems encountered in distributed computing. However, it lacks a systematic design for the problems it tries to solve, so the solutions are neither consistent nor complete.

For a caching problem inside a workflow, we need to consider several properties of a checkpoint:

* **lazy vs eager**: as mentioned above, lazy checkpoint happens only when the first usage happens and eager checkpoint happens immediately regardless how it will be used in later steps
* **memory vs filesystem**: storing data into the current worker's memory (or worker's local disk) is fast, but it has to keep the linage to deal with failure, plus, it can't cross execution. Using (global) filesystem is slower, but it can break the lineage and may be used cross execution. Also using filesystem, the workers can have more free space for compute.
* **permanent vs non-permanent**: memory must be non-permanent but filesystem can be permanent or non-permanent. For the permanent case, the filesystem checkpoint is not removed after one execution. For the non-permanent case, the checkpoint files will be removed after workflow execution finished. So only permanent checkpoint can be used cross execution.
* **deterministic vs non-deterministic**: determinism is for cross execution cases, so only for permanent checkpoint, determinism is meaningful. For example for a defined execution graph, in the first execution, it failed after a certain step that has a deterministic checkpoint. If we run the exactly same graph again, due to determinism, we are able to know recover the checkpoint file when running the second time, so we can reliably and immediately 'resume' from the last failure point. For a non-deterministic permanent checkpoint, it is to intentionally disable 'resume' but to let users who get the checkpoint key (like a password token) use that data after execution. 
* **explicit vs implicit**: for a permanent checkpoint, the checkpoint key can be explicit, for example a user specified file path, or implicit, for example a random or deterministic uuid. So explicit checkpoint must also be a deterministic checkpoint, the difference is who defines the checkpoint key.

The following chart can explain the logical relationship between the properties:

<img src="../images/checkpoint.svg" width="500">


## Specializing The Concept

Mapping the properties to Spark and Fugue equivalent, we can have this comparison table

Lazy | Filesystem | Permanent | Deterministic | Explicit | Spark Equivalent | Fugue Equivalent
:---:|:---:|:---:|:---:|:---:|:---|:---
Yes   | No        | No        | No            | No      | `persist()` | `weak_checkpoint(lazy=True)`
No    | No        | No        | No            | No      |  | `persist()` or `weak_checkpoint(lazy=False)`
Yes   | Yes       | No        | No            | No      |  | `strong_checkpoint(lazy=True)`
No    | Yes       | No        | No            | No      |  | `checkpoint()` or `strong_checkpoint(lazy=False)`
Yes   | Yes       | Yes       | No            | No      |  | 
No    | Yes       | Yes       | No            | No      | `checkpoint` | `yield_as`
Yes   | Yes       | Yes       | Yes           | No      | | `deteriministic_checkpoint(lazy=True)`
No    | Yes       | Yes       | Yes           | No      | | `deteriministic_checkpoint()`
Yes   | Yes       | Yes       | Yes           | Yes     |  | 
No    | Yes       | Yes       | Yes           | Yes     | `save` & `read` | `save` & `load`
